In [1]:
import cv2
import dlib
from matplotlib import pyplot as plt
#from imutils import face_utils
import pandas as pd

print("OpenCV version:", cv2.__version__)
print("dlib version:", dlib.__version__)

image = cv2.imread("../data/person.jpg")  # Замените на ваш путь
if image is None:
    raise ValueError("Изображение не загружено!")

print("Размер изображения:", image.shape)  # Должно быть (H, W, 3) для BGR
print("Тип данных:", image.dtype)  # Должно быть uint8

# Загружаем модель
predictor = dlib.shape_predictor("../models/shape_predictor_68_face_landmarks.dat")
detector = dlib.get_frontal_face_detector()

# Преобразуем изображение в оттенки серого
image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Детектируем лицо
faces = detector(image)

data = []
#Для каждого лица найдём ключевые точки
for face in faces:
    landmarks = predictor(image, face)
    #shape = face_utils.shape_to_np(shape)

    for idx in list(range(0, 17)) + list(range(17, 27)):
        x = landmarks.part(idx).x
        y = landmarks.part(idx).y
        data.append({"point_id": idx, "x": x, "y": y})

# Сохраняем в CSV
df = pd.DataFrame(data)
df.to_csv("../face_landmarks.csv", index=False)

print("Сохранено:", df.shape[0], "точек")

# Отображаем результат
# plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
# plt.axis("off")
# plt.show()

# Показ изображения с результатами
# cv2.imshow("Eyebrow Landmarks", image)
# cv2.waitKey(0)
# cv2.destroyAllWindows()





OpenCV version: 4.11.0
dlib version: 19.22.99
Размер изображения: (398, 246, 3)
Тип данных: uint8
Сохранено: 27 точек
